In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import BallTree
import geopandas as gpd
from shapely.geometry import Point, LineString
from pyproj import Proj, transform
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep, SegmentedLargeMultinomialLogitStep
from urbansim.utils import misc
from urbansim_defaults.utils import yaml_to_class
from urbansim.models.util import apply_filter_query
import orca
import os; os.chdir('../')

In [3]:
import warnings;warnings.simplefilter('ignore')
from baus import datasources, models, variables, ual
from choicemodels import MultinomialLogit
from choicemodels.tools import MergedChoiceTable

### Set up Orca environment

In [4]:
orca.add_injectable("year", 2015)

In [5]:
orca.run([
    "generate_skims_vars",
    "skims_aggregations_drive",
    "neighborhood_vars",
    "price_vars",
    "rsh_simulate",
    "wlcm_simulate"
])

Running step 'generate_skims_vars'
Calculating number of households for zones
Calculating mean_persons of households for zones
Calculating median_persons of households for zones
Calculating std_persons of households for zones
Calculating sum_persons of households for zones
Calculating mean_income of households for zones
Calculating median_income of households for zones
Calculating std_income of households for zones
Calculating sum_income of households for zones
Calculating number of jobs for zones
Calculating mean_sector_id of jobs for zones
Calculating median_sector_id of jobs for zones
Calculating std_sector_id of jobs for zones
Calculating number of buildings for zones
Calculating mean_residential_units of buildings for zones
Calculating median_residential_units of buildings for zones
Calculating std_residential_units of buildings for zones
Calculating sum_residential_units of buildings for zones
Time to execute step 'generate_skims_vars': 18.90 s
Running step 'skims_aggregations_dr

Calculating number of households for parcels
Calculating mean_persons of households for parcels
Calculating median_persons of households for parcels
Calculating std_persons of households for parcels
Calculating sum_persons of households for parcels
Calculating mean_income of households for parcels
Calculating median_income of households for parcels
Calculating std_income of households for parcels
Calculating sum_income of households for parcels
Calculating number of jobs for parcels
Calculating mean_sector_id of jobs for parcels
Calculating median_sector_id of jobs for parcels
Calculating std_sector_id of jobs for parcels
Calculating number of buildings for parcels
Calculating mean_residential_units of buildings for parcels
Calculating median_residential_units of buildings for parcels
Calculating std_residential_units of buildings for parcels
Calculating sum_residential_units of buildings for parcels
Calculating number of jobs for zones
Iteration 1: 191347 of 3293834 valid choices
Iter

In [6]:
mm.initialize()

Registering model step 'wlcm'


### Create Merged Choice Table

- Have to merge persons-level attributes (i.e. commutes) AFTER creating the merged choice table on households

In [23]:
chooser_filters = ['tenure == "own"', 'income > 0', 'unit_id >= 0']
alt_sample_size = 50

In [24]:
obs = orca.get_table('households').to_frame().sample(100000)
obs = apply_filter_query(obs, filters=chooser_filters)
obs = obs[['unit_id', 'tenure', 'income']]
obs.index.name = 'obs_id'

In [25]:
alts = orca.merge_tables(
    'residential_units', ['residential_units'] + orca.get_injectable('aggregations') + ['buildings', 'zones'] )

alts.rename(columns={'zone_id': 'zone_id_home'}, inplace=True)

alts = alts[[
    'total_jobs_gen_tt_CAR_45', 'sum_income_gen_tt_CAR_45', 'juris_ave_income', 'jobs_1500',
    'residential_units_1500', 'unit_residential_price', 'sqft_per_unit', 'sfdu', 'zone_id_home'
]]

In [26]:
interaction_terms = orca.get_table('beam_skims_imputed').to_frame().rename_axis(
            ['zone_id_home', 'zone_id_work'])[['dist', 'gen_tt_CAR', 'gen_tt_WALK_TRANSIT']]

interaction_terms = interaction_terms.reset_index().set_index(['zone_id_work', 'zone_id_home'])

In [27]:
mct = MergedChoiceTable(
    obs, alts,
    chosen_alternatives='unit_id',
    sample_size=alt_sample_size)

In [28]:
mct = MergedChoiceTable(
    obs, alts,
    chosen_alternatives='unit_id',
    sample_size=alt_sample_size,
                       )
mct_df = mct.to_frame()
mct_df.reset_index(inplace=True)

In [29]:
persons = orca.get_table('persons').to_frame(columns=['household_id', 'job_id'])
jobs = orca.get_table('jobs').to_frame(columns=['zone_id_work'])
persons_jobs = pd.merge(persons, jobs, left_on='job_id', right_index=True, how='left')

mct_persons = pd.merge(mct_df, persons_jobs, left_on='obs_id', right_on='household_id', how='left')
mct_persons = mct_persons.join(interaction_terms, how='left', on=interaction_terms.index.names)

person_aggs = mct_persons.groupby(['obs_id', 'unit_id']).agg(
    max_hh_commute_auto = pd.NamedAgg('gen_tt_CAR', 'max'),
    max_hh_commute_transit = pd.NamedAgg('gen_tt_WALK_TRANSIT', 'max'),
    max_hh_commute_dist = pd.NamedAgg('dist', 'max')
).reset_index()

In [30]:
mct_df = pd.merge(mct_df, person_aggs, on=['obs_id', 'unit_id'], how='left')
mct_df['segment'] = 'no commuters'
mct_df.loc[~pd.isnull(mct_df['max_hh_commute_auto']), 'segment'] = '1+ commuter'
mct_df.set_index(['obs_id', 'unit_id'], inplace=True)

In [31]:
mct = MergedChoiceTable.from_df(mct_df)

### Fit model

In [32]:
default = LargeMultinomialLogitStep(
    choosers=pd.DataFrame(),
    chooser_filters=chooser_filters,
    choice_column='unit_id',
    constrained_choices=True,
    alternatives=pd.DataFrame(),
    alt_sample_size=alt_sample_size,
)

In [45]:
default.model_expression = "np.log1p(total_jobs_gen_tt_CAR_45) + np.log1p(sum_income_gen_tt_CAR_45) + " + \
    "juris_ave_income + jobs_1500 + residential_units_1500 + np.log1p(unit_residential_price) - 1"

In [53]:
m = SegmentedLargeMultinomialLogitStep(
    defaults=default, 
    segmentation_column='segment'
)

In [55]:
commuter_hlcm = m.submodels['1+ commuter']

In [56]:
commuter_hlcm.model_expression = commuter_hlcm.model_expression + \
    ' + max_hh_commute_auto * np.log1p(max_hh_commute_dist)'

In [57]:
m.fit_all(mct)

################### SEGMENT: segment = 1+ commuter ###################
                  CHOICEMODELS ESTIMATION RESULTS                   
Dep. Var.:                chosen   No. Observations:          35,591
Model:         Multinomial Logit   Df Residuals:              35,582
Method:       Maximum Likelihood   Df Model:                       9
Date:                 2020-04-13   Pseudo R-squ.:              0.013
Time:                      20:55   Pseudo R-bar-squ.:          0.013
AIC:                 274,778.938   Log-Likelihood:      -137,380.469
BIC:                 274,855.256   LL-Null:             -139,232.811
                                                       coef   std err         z     P>|z|   Conf. Int.
------------------------------------------------------------------------------------------------------
np.log1p(total_jobs_gen_tt_CAR_45)                   0.2010     0.030     6.709     0.000             
np.log1p(sum_income_gen_tt_CAR_45)                  -0.1807     0.02